In [1]:
import sys

from pathlib import Path
main_path = Path().absolute().parent
data_path = main_path / 'data'
setting_path = main_path / 'setting_files'

sys.path.append(str(main_path))

import yaml
import pytorch_lightning as pl
from pytorch_lightning import seed_everything

from src.nlu_models import NLUModel
from src.nlu_utils import NLUDataModule

with (setting_path / 'eval_settings.yml').open('r') as file:
    settings = yaml.load(file, Loader=yaml.FullLoader)

data_module = NLUDataModule(
    train_path=data_path / settings['train_file'], 
    valid_path=data_path / settings['valid_file'],
    test_path=data_path / settings['test_file'],
    labels_path=data_path / settings['labels_file'],
    batch_size=settings['batch_size'], 
    max_len=settings['max_len'],
    num_workers=settings['num_workers'],
    seed=settings['seed']
)

hparams = {
    'stage': settings['stage'],
    'model_path': settings['model_path'], 
    'intent_size': len(data_module.intents2id), 
    'tags_size': len(data_module.tags2id), 
    'max_len': settings['max_len'],
    'lr': settings['lr'],
    'multigpu': True if settings['n_gpus'] > 1 else False
}

checkpoint_path = str(main_path / 'checkpoints' / settings['exp_name'] / 'best_model.ckpt')

model = NLUModel.load_from_checkpoint(checkpoint_path)
model.eval()

seed_everything(seed=settings['seed'])
trainer = pl.Trainer(
    gpus=settings['n_gpus'], 
    max_epochs=settings['n_epochs'], 
    num_sanity_val_steps=settings['num_sanity_val_steps'],
    deterministic=True,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [ ]:
# results = trainer.test(model, ckpt_path=checkpoint_path, datamodule=data_module)

In [2]:
from src.nlu_utils import NLUTokenizer

tokenizer = NLUTokenizer()

In [3]:
def predict(model, tokenizer, text):
    bert_encodes = tokenizer.pipeline_tokenize(text, max_len=256)
    output = model.predict(**bert_encodes)
    return output

In [4]:
test_case = "how is the current liabilities in the latest year?"
o = predict(model, tokenizer, test_case)

In [5]:
o

{'tags': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'intent': tensor([0])}